# geom #1
Cantilever Beam

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
L = 1.0;   E = 200e9;   I = 1e-6;   q = 10000.0
def analytical(x):
    return - (q / (E * I)) * (x**4 / 24 - x**3 / 6 + x**2 / 4)

In [ ]:
geom = dde.geometry.Interval(0, L)

In [ ]:
# ODE
def pde(x ,y):
    dy_xx = dde.grad.hessian(y, x)
    dy_xxxx = dde.grad.hessian(dy_xx, x)
    return dy_xxxx + (q / (E * I))

In [ ]:
# left
def boundary_value_l(x):
    return 0

def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

# right
def ddy(x, y):
    return dde.grad.hessian(y, x)

def dddy(x, y):
    return dde.grad.jacobian(ddy(x, y), x)

def boundary_second_derivative(x, y, _):
    return ddy(x, y)

def boundary_third_derivative(x, y, _):
    return dddy(x, y)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], L)

In [ ]:
bc_dbc = dde.icbc.DirichletBC(geom, boundary_value_l, boundary_l)
bc_nbc = dde.icbc.NeumannBC(geom, boundary_value_l, boundary_l)

bc_obc1 = dde.icbc.OperatorBC(geom, boundary_second_derivative, boundary_r)
bc_obc2 = dde.icbc.OperatorBC(geom, boundary_third_derivative, boundary_r)

In [ ]:
bc = [bc_dbc, bc_nbc, bc_obc1, bc_obc2]
data = dde.data.PDE(geom, pde, bc, 15, 2 , solution=analytical, num_test=100)

In [ ]:
layer_size = [1] + [30]*3 +[1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=4000)

In [ ]:
x_values_pinn = np.linspace(0, L, 100).reshape(-1, 1)
x_values_exact = np.linspace(0, L, 20).reshape(-1, 1)

y_pred = model.predict(x_values_pinn)
y_exact = analytical(x_values_exact)


plt.figure(figsize=(10, 6))
plt.plot(x_values_exact, y_exact, label="Exact Solution", color="blue")
plt.plot(x_values_pinn, y_pred, label="PINN Prediction", color="red", linestyle="--")
plt.xlabel("x")
plt.ylabel("u(x)")
plt.title("PINN vs Analytical: Deflection of Cantilever Beam Under Uniform Load")
plt.legend()
plt.grid()
plt.show